In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df= pd.read_csv("sap_storing_data_hu_project.csv")

In [ ]:
df.head()

haalt de data's uithalen dat de Betekenis een vraagteken of nvt staan.

In [ ]:
df=df[['#stm_sap_meldnr','stm_mon_nr','stm_vl_post','stm_sap_meld_ddt','stm_sap_meldtekst_lang','stm_sap_meldtekst','stm_geo_mld','stm_equipm_nr_mld','stm_equipm_soort_mld','stm_equipm_omschr_mld','stm_km_van_mld',
   'stm_km_tot_mld','stm_prioriteit','stm_aanngeb_ddt','stm_oh_pg_gst','stm_geo_gst','stm_equipm_nr_gst','stm_equipm_soort_gst','stm_equipm_omschr_gst','stm_km_van_gst','stm_km_tot_gst','stm_oorz_groep','stm_oorz_code',
   'stm_oorz_tkst','stm_fh_ddt','stm_fh_status','stm_sap_storeind_ddt','stm_tao_indicator','stm_tao_indicator_vorige','stm_tao_soort_mutatie','stm_tao_telling_mutatie','stm_tao_beinvloedbaar_indicator','stm_sap_melddatum',
   'stm_sap_meldtijd','stm_contractgeb_mld','stm_techn_mld','stm_techn_gst','stm_aanngeb_dd','stm_aanngeb_tijd','stm_aanntpl_dd','stm_aanntpl_tijd','stm_progfh_in_datum','stm_progfh_in_tijd','stm_progfh_in_invoer_dat',
   'stm_progfh_in_invoer_tijd','stm_progfh_in_duur','stm_progfh_gw_tijd','stm_progfh_gw_teller','stm_fh_dd','stm_fh_tijd','stm_fh_duur','stm_sap_storeinddatum','stm_sap_storeindtijd','stm_oorz_tekst_kort',
   'stm_pplg_van','stm_pplg_naar','stm_dstrglp_van','stm_dstrglp_naar']]

In [ ]:
df.info()

In [ ]:
df2=df.dropna(subset=['stm_oorz_code'])
df2['stm_oorz_code'] = df2['stm_oorz_code'].astype(int)

filter functiehersteltijd met bijhoorde oorzaak code voor de baseline model 

In [ ]:
df2[['stm_oorz_code','stm_sap_meld_ddt','stm_fh_dd','stm_fh_tijd','stm_fh_duur']]

In [ ]:
df2[['stm_oorz_code','stm_sap_meld_ddt','stm_fh_dd','stm_fh_tijd','stm_fh_duur']].info()

In [ ]:
df2 = df2.dropna(subset=['stm_fh_duur'])
df2[['stm_oorz_code','stm_sap_meld_ddt','stm_fh_dd','stm_fh_tijd','stm_fh_duur']].info()

In [ ]:
df_gm=df2.groupby('stm_oorz_code')['stm_fh_duur'].agg(['mean'])
df_value=df_gm['mean'].to_list()
df_key=df2['stm_oorz_code'].unique()

In [ ]:
dictionary = dict(zip(df_key, df_value))

In [ ]:
list=[]
def percentage_deviation(value, reference):
    if reference == 0:
        return float('inf') 
    return ((value - reference) / reference) * 100

for index, row in df2.iterrows():
    key = row["stm_oorz_code"]
    value_in_dict = dictionary.get(key, None)
    
    if value_in_dict is not None:
        deviation = percentage_deviation(row["stm_fh_duur"], value_in_dict)
        list.append(abs(deviation))
np.mean(list)